In [42]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
# Paths to your static datasets
elevation_path = 'calraster/elevation.tif'
soil_path = 'calraster/soil_kozhikode.tif'
slope_path = 'calraster/slope_kozhikode.tif'
landuse_path = 'calraster/Landuse_kozhikode.tif'
def reproject_and_resample_raster(src_path, dst_path, reference_raster_path):
    with rasterio.open(reference_raster_path) as ref_src:
        transform, width, height = calculate_default_transform(
            ref_src.crs, ref_src.crs, ref_src.width, ref_src.height, *ref_src.bounds
        )
        kwargs = ref_src.meta.copy()
        kwargs.update({
            'crs': ref_src.crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(src_path) as src:
            with rasterio.open(dst_path, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=ref_src.crs,
                        resampling=Resampling.bilinear
                    )

# Paths to the original and reprojected rasters
reprojected_elevation_path = 'calraster/reprojected_elevation.tif'
reprojected_soil_path = 'calraster/reprojected_soil.tif'
reprojected_slope_path = 'calraster/reprojected_slope.tif'
reprojected_landuse_path = 'calraster/reprojected_landuse.tif'

# Reproject and resample rasters to match the elevation raster
reproject_and_resample_raster(elevation_path, reprojected_elevation_path, elevation_path)  # Use elevation as the reference
reproject_and_resample_raster(soil_path, reprojected_soil_path, elevation_path)
reproject_and_resample_raster(slope_path, reprojected_slope_path, elevation_path)
reproject_and_resample_raster(landuse_path, reprojected_landuse_path, elevation_path)


In [46]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

files = glob.glob("calraster/regridded/*.npy")
# print(len(files))

for filename in files:
    temp = np.load(filename)
    temp = temp.reshape(-1)
    print(pd.DataFrame(temp).describe())

         0
count  0.0
mean   NaN
std    NaN
min    NaN
25%    NaN
50%    NaN
75%    NaN
max    NaN
         0
count  0.0
mean   NaN
std    NaN
min    NaN
25%    NaN
50%    NaN
75%    NaN
max    NaN
                  0
count  1.391244e+06
mean  -1.210890e+04
std    1.583220e+04
min   -3.276800e+04
25%   -3.276800e+04
50%    1.500000e+01
75%    2.300000e+01
max    9.700000e+01
                  0
count  1.391244e+06
mean   1.266790e+38
std    1.643877e+38
min   -1.776357e-15
25%    3.576017e+00
50%    1.002412e+01
75%    3.400000e+38
max    3.400000e+38
